In [11]:
import duckdb
from pathlib import Path
from dotenv import load_dotenv
import os

In [12]:
dotenv_path = Path("/Users/arthurgonzaga/DataKick/.env")
load_dotenv(dotenv_path=dotenv_path)

True

In [13]:

# Configuração das credenciais do MinIO
MINIO_ACCESS_KEY = os.getenv('MINIO_ACCESS_KEY')
MINIO_SECRET_KEY = os.getenv('MINIO_SECRET_KEY')
MINIO_ENDPOINT = "localhost:9000"

# Caminho do arquivo no MinIO
minio_csv_path = "s3://bronze/datadrop/kaggle/database.csv"

In [14]:

# Conectando ao DuckDB
con = duckdb.connect()

config = f"""
INSTALL httpfs;
LOAD httpfs;
SET s3_access_key_id='{MINIO_ACCESS_KEY}';
SET s3_secret_access_key='{MINIO_SECRET_KEY}';
SET s3_endpoint='{MINIO_ENDPOINT}'; -- Endpoint do MinIO
SET s3_url_style = 'path';
SET s3_use_ssl=false; -- Defina como 'true' se estiver usando SSL/TLS no MinIO
"""

# Configurando as credenciais e o endpoint do MinIO no DuckDB
con.execute(config)

In [15]:
print(config)


INSTALL httpfs;
LOAD httpfs;
SET s3_access_key_id='jknVRF5166kyQRLiK7gc';
SET s3_secret_access_key='OKczJuafJxBfAQ8xn9hiHguhbQtQgmYALDwLY1Sy';
SET s3_endpoint='localhost:9000'; -- Endpoint do MinIO
SET s3_url_style = 'path';
SET s3_use_ssl=false; -- Defina como 'true' se estiver usando SSL/TLS no MinIO



In [21]:
query = f"""
SELECT
    Jogador AS player,
    Time AS team,
    "#" AS number,
    Nação AS nationality,
    "Pos." AS position,
    Idade AS age,
    "Min." AS minutes,
    Gols AS goals,
    "Assis." AS assists,
    PB AS progressive_passes,
    PT AS total_passes,
    TC AS tackles,
    CaG AS goals_conceded,
    CrtsA AS yellow_cards,
    CrtV AS red_cards,
    Contatos AS touches,
    Div AS duels,
    Crts AS fouls,
    Bloqueios AS blocks,
    xG AS expected_goals,
    npxG AS non_penalty_expected_goals,
    xAG AS expected_assists,
    SCA AS shot_creating_actions,
    GCA AS goal_creating_actions,
    Cmp AS passes_completed,
    Att AS passes_attempted,
    "Cmp%" AS pass_completion_percentage,
    PrgP AS progressive_passes_distance,
    Conduções AS dribbles,
    PrgC AS progressive_carries,
    Tent AS dribbles_attempted,
    Suc AS dribbles_successful,
    Data AS updated_date
FROM read_csv_auto('{minio_csv_path}')
"""

In [22]:
# Carregando o CSV do MinIO para uma tabela temporária e lendo os dados
try:
    result = con.execute(query).df()
    print(result.head())  # Mostra as primeiras linhas do dataframe
except Exception as e:
    print("Erro ao acessar o MinIO:", e)

               player      team  number nationality position     age  minutes  \
0                Éder  Criciúma      23         ITA       FW  37-150     57.0   
1     Yannick Bolasie  Criciúma      11         COD       FW  34-325     33.0   
2       Renato Kayser  Criciúma      79         BRA       FW  28-056     85.0   
3              Arthur  Criciúma      35         BRA       FW  31-303      5.0   
4  Marquinhos Gabriel  Criciúma      10         BRA       AM  33-267     77.0   

   goals  assists  progressive_passes  ...  goal_creating_actions  \
0      0        0                   0  ...                    0.0   
1      0        0                   0  ...                    0.0   
2      1        0                   0  ...                    0.0   
3      0        0                   0  ...                    0.0   
4      0        0                   0  ...                    1.0   

   passes_completed  passes_attempted  pass_completion_percentage  \
0              12.0          

In [18]:
# Fechar a conexão
# con.close()